# Tarea 4. Modelos de Lenguaje Estadísticos

Guillermo Segura Gómez

## Modelos de Lenguaje y Evaluación
**1. Preprocese todos los tuits de agresividad (positivos y negativos) según su intuición para construir un buen corpus para un modelo de lenguaje (e.g., solo palabras en minúscula, etc.). Agregue tokens especiales de <s> y </s> según usted considere (e.g., al inicio y final de cada tuit). Defina su vocabulario y enmascare con <unk> toda palabra que no esté en su vocabulario.**

### Modelos de Lenguaje

Un modelo de lenguaje típicamente asigna probabilidades a secuencias de palabras o genera secuencias de palabras basándose en las probabilidades aprendidas. Estas probabilidades reflejan qué tan probable es que una palabra siga a otra u otras en una secuencia. Los modelos de lenguaje pueden ser de varios tipos, incluidos los basados en reglas gramaticales, los estadísticos, y más recientemente, los basados en redes neuronales, como los modelos de lenguaje transformadores.

### Modelos de Lenguaje de N-gramas
Los modelos de lenguaje de n-gramas son un tipo específico de modelo de lenguaje estadístico que se basa en la premisa de que la probabilidad de una palabra en una secuencia depende solo de las $n-1$ palabras anteriores. Un "n-grama" es, por lo tanto, una secuencia de $n$ elementos (por ejemplo, palabras) del texto. Por ejemplo:

- Un 1-grama (o unigrama) considera cada palabra de forma aislada.
- Un 2-grama (o bigrama) considera pares de palabras consecutivas.
- Un 3-grama (o trigrama) considera tríos de palabras consecutivas, y así sucesivamente.

En un modelo de n-gramas, la probabilidad de una palabra dada está condicionada por las $n-1$ palabras anteriores. Estos modelos son relativamente simples de construir y pueden ser muy efectivos para ciertas tareas, especialmente cuando los recursos computacionales son limitados o cuando se dispone de una cantidad limitada de datos de entrenamiento. Sin embargo, los modelos de n-gramas tienen limitaciones, como la dificultad para manejar contextos más largos y el problema de la "explosión combinatoria" de posibles n-gramas a medida que $n$ aumenta, lo que puede hacer que el modelo sea menos práctico para valores grandes de $n$.

Para poder construir un modelo de n-gramas primero necesitamos pre procesar nuestro corpus para que sea mas sencillo de manejar. Por ejemplo, podemos trabajar únicamente palabras en minúscula. Además, es necesario tener delimitadores de oraciones \<s\> y \</s\> y definir un vocabulario. Cualquier palabra fuera del vocabulario le asignamos el token \<unk\>. Vamos a trabajar con los tweets de agresividad del corpus [MEX-A3T](https://sites.google.com/view/mex-a3t/home?authuser=0) con el que hemos estado trabajando.

In [3]:
# Función que extrae el texto de dos archivos. Uno el de los documentos, otro el de las etiquetas
def get_text_from_file(path_corpus, path_truth):

    tr_text = []
    tr_labels = []

    with open(path_corpus, "r") as f_corpus, open(path_truth, "r") as f_truth:
        for tweet in f_corpus:
            tr_text += [tweet]
        for label in f_truth:
            tr_labels += [label]

    return tr_text, tr_labels

In [4]:
path_text = "/home/guillermosegura/Desktop/Segundo Semestre/Natural-Language-Processing/MexData/mex20_train.txt"
path_labels = "/home/guillermosegura/Desktop/Segundo Semestre/Natural-Language-Processing/MexData/mex20_train_labels.txt"

path_text_val = "/home/guillermosegura/Desktop/Segundo Semestre/Natural-Language-Processing/MexData/mex20_val.txt"
path_labels_val = "/home/guillermosegura/Desktop/Segundo Semestre/Natural-Language-Processing/MexData/mex20_val_labels.txt"

tr_text, tr_labels = get_text_from_file(path_text, path_labels) # Importamos los datos de entrenamiento
val_text, val_labels = get_text_from_file(path_text_val, path_labels_val) # Importamos los datos de test o validación

Lo primero que tenemos que hacer es delimitar nuestro vocabulario. Limitamos solo las palabras en minúsculas. 
Además debemos agregar los tokens de inicio y final a cada documento del corpus.

In [5]:
import nltk
from nltk.tokenize import TweetTokenizer
import re

In [6]:
# Función para preprocesar los textos
def preprocess_texts(texts):
    # Convertir a minúsculas y eliminar números y caracteres especiales
    texts_lower = [re.sub(r"http\S+|[^a-zA-Z\s]", "", doc.lower()) for doc in texts]
    return texts_lower

# Función que agrega delimitadores de oraciones a un corpus
def addlimiters(texts):
    texts_limites = []
    for doc in texts:
        newText = "<s>" + doc + "</s>"
        texts_limites.append(newText)

    return texts_limites

# Preprocesar el texto
tr_text_min = preprocess_texts(tr_text)

# Agregar delimitadores de oración
tr_text_limited = addlimiters(tr_text_min)


Ahora tokenizamos el texto.

In [7]:
tokenizer = TweetTokenizer() # Inicializar tokenizer
corpus_palabras = []

for doc in tr_text_limited:
    corpus_palabras += tokenizer.tokenize(doc)

fdist = nltk.FreqDist(corpus_palabras)

print(f"El tamaño del corpus es:", len(corpus_palabras))
print(f"El tamaño del vocabulario es:", len(fdist))

El tamaño del corpus es: 94261
El tamaño del vocabulario es: 12388


Definimos una función para construir el vocabulario. Utilizamos una longitud del vocabulario de cinco mil. 

In [8]:
# Función para ordenar las frecuencias
def  SortFrecuency(freqdist):
    # List comprenhension
    aux = [(freqdist[key], key) for key in freqdist]
    aux.sort() # Ordena la lista
    aux.reverse() # Cambiar el orden

    return aux

# Ordenamos y obtenemos el vocabulario
voc = SortFrecuency(fdist)
voc = voc[:5000]
voc[:5]

[(5278, '<s>'), (5278, '</s>'), (3102, 'que'), (3095, 'de'), (2268, 'la')]

Ahora reemplazamos las palabras desconocidas con el token \<unk\>

In [9]:
# Función para reemplazar palabras fuera del vocabulario con <unk>
def replace_unknowns(tokenized_texts, vocab):
    # Reemplazar palabras que no están en el vocabulario con <unk>
    return [[word if word in vocab else '<unk>' for word in doc] for doc in tokenized_texts]

tr_text_final = replace_unknowns(corpus_palabras, voc)

In [10]:
tr_text_final[:5]

[['<unk>', '<unk>', '<unk>'],
 ['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>'],
 ['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>'],
 ['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>'],
 ['<unk>']]

**2. Entrene tres modelos de lenguaje sobre todos los tuits: $P_{unigramas}(w_1^n)$, $P_{bigramas}(w_1^n)$, $P_{trigramas}(w_1^n)$. Para cada uno proporcione una interfaz (función) sencilla para  $P_{n-grama}(w_1^n)$ y  $P_{n-grama}(w_1^n | w_{n-N+1}^{n-1})$.  Los modelos deben tener una estrategia común para lidiar con secuencias no vistas. Puede optar por un suavizamiento Laplace o un Good-Turing discounting. Muestre un par de ejemplos de como funciona, al menos uno con una palabra fuera del vocabulario.**

El modelo de n-gramas que vamos a construir es una función que recibe una cadena de caracteres y asigna la probabilidad en base a las $n-1$ palabras de las que este formada la cadena. Para ejemplificar esto, consideremos el ejemplo mostrado en el capitulo 3 del libro de Daniel Jurafsky [1].

En el capitulo 3 del libro se busca encontrar un modelo que calcule la probabilidad de que la siguiente palabra a la oración *“
its water is so transparent that* sea *the* es decir

$$
P(\text{the}|\text{its water is so transparent that})
$$

* En un modelo de unigramas se asume que la aparición de cada palabra es independiente de las palabras antes o después de ella en la secuencia. Esto significa que no se considera el contexto en el que aparece la palabra; cada palabra es tratada de forma aislada.

    La probabilidad de una secuencia de palabras $ w_1^n $ en un modelo de unigramas se calcula simplemente como el producto de las probabilidades individuales de cada palabra en la secuencia:

    $$ P_{\text{unigramas}}(w_1^n) = P(w_1) \times P(w_2) \times \ldots \times P(w_n) = \prod_{i=1}^{n} P(w_i) $$

    Donde:
    - $ P_{\text{unigramas}}(w_1^n) $ es la probabilidad de la secuencia de palabras $ w_1, w_2, \ldots, w_n $ bajo el modelo de unigramas.
    - $ P(w_i) $ es la probabilidad de la palabra individual $ w_i $.

    La probabilidad $P(w_i)$ de cada palabra individual se estima generalmente a partir del corpus de entrenamiento. La forma más simple de estimar $P(w_i)$ es usar la frecuencia relativa de la palabra en el corpus:

    $$ P(w_i) = \frac{\text{Frecuencia de } w_i}{\text{Número total de palabras en el corpus}} $$

* En el modelo de bigramas, calculamos la probabilidad según $n-1$ palabras, es decir

    $$
    P(\text{the}|\text{that})
    $$

    La suposición de que la probabilidad de una palabra depende solo en la palabra previa se conoce como **suposición de Markov**. Para calcular la probabilidad en un modelo de bigramas de una palabra $w_n$ dada una previa palabra $w_{n-1}$ calculamos la cuenta de cuentas ocasiones aparecen las palabras juntas $C(w_{n-1} w_n)$ y la normalizamos por la suma de todos los bigramas que comparten la misma palabra $w_{n-1}$

    $$
    P(w_n|w_{n-1}) = \frac{C(w_{n-1} w_n)}{ \sum_w C(w_{n-1})}
    $$

    Podemos simplificar esta ecuación, ya que la suma de todas las cuentas de los bigramas que empiezan con la misma palabra $w_{n-1}$, debe de ser igual a la cuenta de los unigramas para esta palabra $w_{n-1}$, entonces 

    $$
    P(w_n|w_{n-1}) = \frac{C(w_{n-1} w_n)}{ C(w_{n-1})}
    $$

    Para una secuencia de tres palabras $w_1, w_2, w_3$, la probabilidad bajo el modelo de bigramas sería:

    $$ P_{\text{bigramas}}(w_1, w_2, w_3) = P(w_1) \times P(w_2 | w_1) \times P(w_3 | w_2) $$


Podemos realizar el ejemplo del libro para un corpus limitado de tres frases.

In [11]:
tweets_example = ['I am Sam', 'Sam I am', ' I do not like green eggs and ham']
# Agregar delimitadores de oración
tweets_example_limited = addlimiters(tweets_example)
print(tweets_example_limited)

['<s>I am Sam</s>', '<s>Sam I am</s>', '<s> I do not like green eggs and ham</s>']


### Unigramas

Construimos un vocabulario para unigramas. El vocabulario es basicamente el mismo que hemos manejado siempre. 

In [12]:
tokenizer = TweetTokenizer() # Inicializar tokenizer
corpus_palabras_example = []

for doc in tweets_example_limited:
    corpus_palabras_example += tokenizer.tokenize(doc)

fdist_example = nltk.FreqDist(corpus_palabras_example)

print(f"El tamaño del corpus es:", len(corpus_palabras_example))
print(f"El tamaño del vocabulario es:", len(fdist_example))

El tamaño del corpus es: 20
El tamaño del vocabulario es: 12


Ahora construimos un diccionario limitado para el uso de las palabras. Por ser un ejemplo, tomaremos todo el corpus, pero para cuando utilicemos el corpus de agresividad, se vera la utilidad. 

In [84]:
# Funcion que construye un diccionario limitado a n palabras
def DicLimited(fdist, n):
    # Obtener las n palabras más comunes
    voc = fdist.most_common(n)

    # Construir el diccionario directamente a partir de las n palabras más comunes
    dict_indices = {word: freq for word, freq in voc}

    return dict_indices

Comenzamos calculando la probabilidad de las cadenas de unigramas, es decir la función para $P_{unigramas}(w_1^n)$. El objeto `fdist_example` es un objeto de la clase *nltk.probability.FreqDist*. Este objeto tiene la característica de que almacena la distribución de frecuencias del corpus tokenizado. Entonces si accedemos a el podemos obtener la frecuencia general de las palabras. 

Ademas, es necesario tener una estrategia para manejar los casos en los que tenemos una secuencia cuya alguna de las palabras no estan en el vocabulario ya que por producto de probabilidad, tendremos probabilidad cero, lo cual tiene que ser "suavizado" de alguna forma. Podemos utilizar un suavizamiento *Laplace* o un *Good-Turing discounting*.

Los suavizados de Laplace y Good-Turing son técnicas utilizadas en modelos de lenguaje, como los de n-gramas, para manejar el problema de las probabilidades cero para n-gramas no observados en el conjunto de datos de entrenamiento. Ambas técnicas ajustan las probabilidades estimadas para asignar alguna probabilidad a los n-gramas no observados, mejorando la generalización del modelo a datos no vistos previamente.

#### Suavizado de Laplace
El suavizado de Laplace, es una técnica simple pero efectiva que consiste en agregar un pequeño valor positivo (usualmente 1, pero puede ser otro valor) a los conteos de todas las posibles palabras del vocabulario, incluidas las que no aparecen en el conjunto de datos de entrenamiento. Esto asegura que ninguna palabra tendrá una probabilidad estimada de cero.

Por ejemplo, en un modelo de bigramas, el suavizado de Laplace ajustaría la estimación de la probabilidad condicional de una palabra $w_n$ dada la palabra anterior $w_{n-1}$ como sigue:

$$ P(w_n|w_{n-1}) = \frac{C(w_{n-1} w_n) + 1}{C(w_{n-1}) + V} $$

Donde:
- $C(w_{n-1} w_n)$ es el conteo original del bigrama $(w_{n-1}, w_n)$.
- $C(w_{n-1})$ es el conteo total de bigramas que comienzan con $w_{n-1}$.
- $V$ es el tamaño del vocabulario, es decir, el número de palabras únicas en el conjunto de entrenamiento.
- El valor 1 es el valor de suavizado aditivo, y se agrega tanto al numerador como al denominador multiplicado por el tamaño del vocabulario.

#### Suavizado Good-Turing
El suavizado Good-Turing es una técnica más sofisticada que ajusta los conteos de los n-gramas observados para estimar mejor las probabilidades de los n-gramas no observados. La idea es reducir la probabilidad asignada a los n-gramas que se han visto una o pocas veces y redistribuir esa probabilidad a los n-gramas no observados.

La corrección de Good-Turing ajusta el conteo $C^*$ de un n-grama observado con frecuencia $r$ como sigue:

$$ C^*(r) = (r+1) \frac{N_{r+1}}{N_r} $$

Donde:
- $r$ es el conteo original de un n-grama.
- $N_r$ es el número de n-gramas que aparecen exactamente $r$ veces en el corpus.
- $N_{r+1}$ es el número de n-gramas que aparecen exactamente $r+1$ veces.

Este método tiene la particularidad de que cuando $r=0$ (es decir, para n-gramas no observados), el ajuste $C^*(0)$ da una estimación de la probabilidad para estos casos basada en la proporción de n-gramas que solo aparecen una vez en el corpus.

In [36]:
# Probabilidad de una cadena con un modelo de unigramas.
"""
La función recibe:
secuencia: Cadena de caracteres
unigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de cada palabra unigrama
lenCorpus: Longitud total del corpus tokenizado
"""
def ProbUnigram(secuencia, unigramas, lenCorpus):

    # Tokenizamos la cadena
    tokens = tokenizer.tokenize(secuencia)

    prob = 1.0 # Inicializamos la probabilidad

    # Tamaño del vocabulario
    V = len(unigramas)

    for word in tokens:
        # Calculo de la probabilidad
        # Suavizado de Laplace: agregamos 1 al conteo de la palabra y V al total del corpus
        word_count = unigramas[word] + 1
        prob *= word_count / (lenCorpus + V)

    return prob

Probamos la funcion. La funcion funciona con unigramas individuales y tambien con cadenas de caracteres debido a que tokenizamos dentro de la funcion

In [88]:
# Construimos un diccionario con las palabras mas frecuentes
unigramas = DicLimited(fdist_example, 50)

In [89]:
# Probabilidad de un unigrama
w1 = "I"
print(f"Probabilidad de \"{w1}\" en un modelo de unigramas: ", ProbUnigram(w1, unigramas, len(corpus_palabras_example)))

# Probabilidad de una secuencia
secuencia = "<s> I am Sam </s>"
print(f"Probabilidad de \"{secuencia}\" en un modelo de unigramas: ", ProbUnigram(secuencia, unigramas, len(corpus_palabras_example)))

Probabilidad de "I" en un modelo de unigramas:  0.18181818181818182
Probabilidad de "<s> I am Sam </s>" en un modelo de unigramas:  0.00011176583815064792


### Bigramas

Ahora, para realizar la función $P_{bigramas}(w_1^n)$ necesitamos construir un vocabulario para bigramas.  

In [15]:
from nltk import bigrams

# Realizamos el mismo proceso que con los unigramas
# Utilizamos el recurso bigrams de la libreria nltk
corpus_palabras_exampleBigrams = []

for doc in tweets_example_limited:
    corpus_palabras_example = tokenizer.tokenize(doc)
    corpus_palabras_exampleBigrams += bigrams(corpus_palabras_example, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')

fdist_example_Bigramas = nltk.FreqDist(corpus_palabras_exampleBigrams)

print(f"El tamaño del corpus es:", len(corpus_palabras_exampleBigrams))
print(f"El tamaño del vocabulario es:", len(fdist_example_Bigramas))

El tamaño del corpus es: 23
El tamaño del vocabulario es: 17


In [16]:
fdist_example_Bigramas["<s>", "I"]

2

In [57]:
# Probabilidad de una cadena con un modelo de bigramas.
"""
La función recibe:
secuencia: Cadena de caracteres 
unigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de cada palabra unigrama
bigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de bigramas
"""
def ProbBigramas(secuencia, bigramas, unigramas):

    # Tokenizamos la cadena
    tokens = tokenizer.tokenize(secuencia)

    # Inicializamos la probabilidad
    prob = 1.0

    # Tamaño del vocabulario
    V = len(unigramas)
    
    # Generar bigramas para la secuencia dada
    secuencia_bigramas = bigrams(tokens)
    
    for bigrama in secuencia_bigramas:
        
        w1, w2 = bigrama
        # Agregamos suavizado de Laplace
        # Contar el bigrama actual y el unigrama para la palabra anterior
        bigrama_count = bigramas[bigrama] + 1
        unigrama_count = unigramas[w1] + V
        
        # Calcular la probabilidad condicional, evitando la división por cero
        prob_condicional = bigrama_count / unigrama_count
        
        # Multiplicar la probabilidad acumulada
        prob *= prob_condicional
        
    return prob


Probamos la funcion con los ejemplos del libro. Encontramos los mismos valores que en el libro cuando no agregamos el suavizado, por lo que corroboramos que la funcion es correcta o al menos vamos por buen camino. La funcion funciona para secuencias de palabras y dos palabras condicionales, lo cual implica tener una sola funcion para $P_{n-grama}(w_1^n)$ y $P_{n-grama}(w_1^n | w_{n-N+1}^{n-1})$.

In [60]:
# Probabilidad condicional P(I|<s>)
secuencia = "<s> I"
print(f"Probabilidad de \"{secuencia}\" en un modelo de bigramas: ", ProbBigramas(secuencia, fdist_example_Bigramas, fdist_example))

# Probabilidad de una secuencia
secuencia = "<s> I am Sam </s>"
print(f"Probabilidad de \"{secuencia}\" en un modelo de bigramas: ", ProbBigramas(secuencia, fdist_example_Bigramas, fdist_example))

Probabilidad de "<s> I" en un modelo de bigramas:  0.2
Probabilidad de "<s> I am Sam </s>" en un modelo de bigramas:  0.0008163265306122449


### Trigramas

Ahora, para realizar la función $P_{trigramas}(w_1^n)$ construimos un vocabulario para trigramas.  


In [47]:
from nltk import trigrams

# Realizamos el mismo proceso que con los unigramas y bigramas
corpus_palabras_exampleTrigrams = []

for doc in tweets_example_limited:
    corpus_palabras_example = tokenizer.tokenize(doc)
    corpus_palabras_exampleTrigrams += trigrams(corpus_palabras_example, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')

fdist_example_Trigramas = nltk.FreqDist(corpus_palabras_exampleTrigrams)

print(f"El tamaño del corpus es:", len(corpus_palabras_exampleTrigrams))
print(f"El tamaño del vocabulario es:", len(fdist_example_Trigramas))

El tamaño del corpus es: 26
El tamaño del vocabulario es: 21


In [54]:
fdist_example_Trigramas["<s>", "I", "am"]

1

Para la funcion de trigramas el proceso es basicamente el mismo. Solo que ahora tenemos que calcular $P_{\text{trigrama}}(w_n | w_{n-2}, w_{n-1})$, que es la probabilidad de la palabra $w_n$ dadas las dos palabras anteriores $w_{n-2}$ y $w_{n-1}$ que calculamos con las cuentas de los bigramas. 

In [58]:
# Probabilidad de una cadena con un modelo de bigramas.
"""
La función recibe:
secuencia: Cadena de caracteres 
unigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de cada palabra unigrama
bigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de bigramas
trigramas: Un objeto tipo nltk.probability.FreqDist para calcular la frecuencia de trigramas
"""
def ProbTrigramas(secuencia, trigramas, bigramas, unigramas):

    # Tokenizamos la cadena
    tokens = tokenizer.tokenize(secuencia)

    # Inicializamos la probabilidad
    prob = 1.0

    # Tamaño del vocabulario
    V = len(unigramas)
    
    # Generar bigramas para la secuencia dada
    secuencia_trigramas = trigrams(tokens)
    
    for trigrama in secuencia_trigramas:
        
        w1, w2, w3 = trigrama
        # Agregamos suavizado de Laplace
        # Contar el trigrama actual y el bigrama para las dos palabras anteriores
        trigrama_count = trigramas[trigrama] + 1
        bigrama_count = bigramas[(w1, w2)] + V 
        
        # Calcular la probabilidad condicional
        prob_condicional = trigrama_count / bigrama_count
        
        # Multiplicar la probabilidad acumulada
        prob *= prob_condicional
        
    return prob


In [59]:
# Probabilidad condicional P(am|<s> I)
secuencia = "<s> I am"
print(f"Probabilidad de \"{secuencia}\" en un modelo de trigramas: ", ProbTrigramas(secuencia, fdist_example_Trigramas, fdist_example_Bigramas, fdist_example))

# Probabilidad de una secuencia
secuencia = "<s> I am Sam </s>"
print(f"Probabilidad de \"{secuencia}\" en un modelo de trigramas: ", ProbTrigramas(secuencia, fdist_example_Trigramas, fdist_example_Bigramas, fdist_example))

Probabilidad de "<s> I am" en un modelo de trigramas:  0.14285714285714285
Probabilidad de "<s> I am Sam </s>" en un modelo de trigramas:  0.003139717425431711


**3. Construya un modelo interpolado con valores $\lambda$ fijos**
$$
\hat{P}\left(w_n \mid w_{n-2} w_{n-1}\right)=\lambda_1 P\left(w_n \mid w_{n-2} w_{n-1}\right)+\lambda_2 P\left(w_n \mid w_{n-1}\right)+\lambda_3 P\left(w_n\right)
$$

**Para ello experimente con el modelo en particiones estratificadas de $80 \%, 10 \%$ y $10 \%$ para entrenar (train), ajuste de parámetros ( $v a l$ ) y prueba (test) respectivamente. Muestre como bajan o suben las perplejidades en validación, finalmente pruebe una vez en test. Para esto puede explorar algunos valores $\vec{\lambda}$ y elija el mejor. Pruebe las siguientes: $[1 / 3,1 / 3,1 / 3],[.4, .4, .2],[.2, .4, .4],[.5, .4, .1]$ y $[.1, .4, .5]$.**


La interpolación nos permite combinar las probabilidades de estos diferentes modelos para obtener una estimación más robusta de la probabilidad de una palabra dada su historia.

El modelo interpolado se define mediante la fórmula dada:

$$ \hat{P}\left(w_n \mid w_{n-2}, w_{n-1}\right) = \lambda_1 P\left(w_n \mid w_{n-2}, w_{n-1}\right) + \lambda_2 P\left(w_n \mid w_{n-1}\right) + \lambda_3 P\left(w_n\right) $$

donde:
- $P\left(w_n \mid w_{n-2}, w_{n-1}\right)$ es la probabilidad de un trigrama (modelo de trigramas).
- $P\left(w_n \mid w_{n-1}\right)$ es la probabilidad de un bigrama (modelo de bigramas).
- $P\left(w_n\right)$ es la probabilidad de un unigrama (modelo de unigramas).
- $\lambda_1, \lambda_2, \lambda_3$ son parámetros que indican el peso de cada modelo en la interpolación. Deben sumar 1 ($\lambda_1 + \lambda_2 + \lambda_3 = 1$) para que la probabilidad total esté bien normalizada.

Tenemos ademas que dividir el conjunto de datos en tres partes: 80% para entrenamiento, 10% para ajuste de parámetros (validación) y 10% para prueba (test). La idea es entrenar el modelo en el conjunto de entrenamiento, ajustar los valores de $\lambda$ en el conjunto de validación para minimizar la perplejidad, y finalmente evaluar el rendimiento del modelo en el conjunto de prueba con el conjunto de $\lambda$ elegido.

La perplejidad es una medida común para evaluar modelos de lenguaje. Una perplejidad más baja indica un mejor modelo de lenguaje, ya que significa que el modelo está menos "sorprendido" por las palabras que ve. 

Lo primero que tenemos que hacer es preprocesar los datos para construir un diccionario como los que hemos realizado en el ejemplo anterior. Dividimos los datos en conjunto de entrenamiento, validacion y prueba. Los datos extraidos del copus de agresividad de Mexico, ya estan divididos en validacion y prueba. Para este ejercicio obviaremos los datos de prueba y utilizaremos el mismo conjunto de entrenamiento subdividio. 

Para dividir los datos en conjuntos de entrenamiento, validación y prueba, utilizamos la función `train_test_split` de la biblioteca `sklearn` dos veces. Primero, para dividir los datos en entrenamiento (80%) y un conjunto temporal (20%), y luego, para dividir ese conjunto temporal en validación y prueba.

### Pasos Sugeridos
3. **Validación**: Para cada conjunto de valores $\lambda$ proporcionado, calcula la perplejidad del modelo en el conjunto de validación y registra los resultados.
4. **Selección de $\lambda$**: Elige el conjunto de $\lambda$ con la menor perplejidad en validación.
5. **Evaluación en Test**: Calcula y reporta la perplejidad del modelo en el conjunto de prueba usando el conjunto de $\lambda$ seleccionado.

Este enfoque te permitirá determinar la mejor combinación de los modelos de unigramas, bigramas y trigramas para tu corpus, balanceando la influencia de cada tipo de contexto (largo, medio y corto alcance) en la estimación de la probabilidad de las palabras.

In [91]:
from sklearn.model_selection import train_test_split

# Primera división: 80% entrenamiento, 20%
train_data, temp_data = train_test_split(tr_text_limited, test_size=0.2, random_state=42)

# Segunda división del conjunto temporal: 50% validación, 50% prueba
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)


Ahora entrenamos los modelos de unigramas, bigramas y trigramas en el conjunto de entrenamiento para obtener las distribuciones de frecuencias necesarias. Para entrenar los modelos lo unico que tenemos que hacer es construir y ajustar los diccioanrios de frecuencias (o distribuciones de frecuencias) para unigramas, bigramas y trigramas.

Hacemos una funcion para esto.

In [95]:
# Funcion que entrena un modelo en base a un conjunto de entrenamiento y un diccionario limitado
def TrainModel(texts, DicSize, model):

    grams = []

    for doc in texts:
        
        # Tokenizamos el documento
        tokens = tokenizer.tokenize(doc)

        if model == "Unigram":
            grams += tokens
        elif model == "Bigram":
            grams += bigrams(tokens, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')
        elif model == "Trigram":
            grams += trigrams(tokens, pad_left=True, pad_right=True, left_pad_symbol='<s>', right_pad_symbol='</s>')
        else:
            print("Modelo no soportado. Por favor, elige Unigram, Bigram o Trigram.")
            return 1

    fdist = nltk.FreqDist(grams)  # Construir FreqDist a partir de la lista de n-gramas

    # Limitamos el diccionario
    limited_dict = DicLimited(fdist, DicSize)

    return limited_dict

In [97]:
# Entrenamos el modelo

unigrams_model = TrainModel(train_data, 5000, "Unigram")
bigrams_model = TrainModel(train_data, 5000, "Bigram")
trigrams_model = TrainModel(train_data, 5000, "Trigram")

## Generación de Texto

Para esta parte reentrenará su modelo de lenguaje interpolado para aprender los valores $\lambda$:
$$
\hat{P}\left(w_n \mid w_{n-2} w_{n-1}\right)=\lambda_1 P\left(w_n \mid w_{n-2} w_{n-1}\right)+\lambda_2 P\left(w_n \mid w_{n-1}\right)+\lambda_3 P\left(w_n\right)
$$

Realice las siguientes actividades:

**1.Proponga una estrategia con base en Expectation Maximization (investigue por su cuenta sobre EM) para encontrar buenos valores de interpolación en $\hat{P}$ usando todo el dataset de agresividad (Se adjunta un material de apoyo). Para ello experimente con el modelo en particiones estratificadas de 80\%, 10\% y 10\% para entrenar (train), ajustar parámetros (val) y probar (test) respectivamente. 1 Muestre como bajan las perplejidades en 5 iteraciones que usted elija (de todas las que sean necesarias de acuerdo a su EM) en validación, y pruebe una vez en test. Sino logra hacer este punto, haga los siguientes dos con el modelo de lenguaje con algunos $\lambda$ fijos.**


## Referencias

[1] Keselj, Vlado. "Book Review: Speech and Language Processing by Daniel Jurafsky and James H. Martin." Computational Linguistics 35.3 (2009).